A project to render a tree with leaves in the wind. Bonus points to render a shadow as well?

Thus far, I've got an extremely simple rigid-body simulation working (Oliapuram & Kumar 2010). Future work would make sure I'm being faithful to their dynamics math, make the code more general, and add support for wind fields.

### Refs

#### Tree animation
Oliapuram & Kumar (2010) Realtime Forest Animation in Wind
Quigley, Yu, Huang, Lin, Fedkiw (2017) Real-time Interactive Tree Animation
Xu, Yang, Ding, Buck-Sorlin (2020) Physics-based algorithm to simulate tree dynamics under wind load

#### Mechanics
Sussman & Wisdom, Structure + Interpretation of Mechanics https://tgvaughan.github.io/sicm/
https://github.com/sicmutils/sicmutils

#### Navier-Stokes (for wind fields)
- https://lorenabarba.com/blog/cfd-python-12-steps-to-navier-stokes/
- https://www.ibiblio.org/e-notes/webgl/gpu/fluid.htm
- http://developer.download.nvidia.com/books/HTML/gpugems/gpugems_ch38.html
- http://jamie-wong.com/2016/08/05/webgl-fluid-simulation/
- https://github.com/amandaghassaei/FluidSimulation


In [20]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import random
%matplotlib inline

In [153]:
# Attempting a first pass at things from Oliapuram & Kumar's Realtime Forest Animation in Wind

def deg_to_rad(deg):
    return deg/360*np.pi*2

preferred = [deg_to_rad(90), deg_to_rad(60), deg_to_rad(120), deg_to_rad(90), deg_to_rad(45), deg_to_rad(90), deg_to_rad(90+45)]
parent = [None, 0, 0, 1, 1, 2, 2] # a simple tree

# Choose some random angles and see how we relax back to preferred angles
angles = (np.array(preferred)*np.random.uniform(0.8, 1.2)).tolist()
velo = [0 for p in preferred]

k = 0.1
kr = 0.5
#mu = 30
mu = 40

history = []
dt = .1
for _ in range(int(60/dt)):
    history.append(tuple(angles))

    n_angles = list(angles)
    n_velo = list(velo)
    rest_child = [0 for _ in preferred]
    rest = 0
    for i in range(len(preferred))[::-1]:
        rest = k * (preferred[i] - angles[i])
        damp = -mu * velo[i] * abs(velo[i])
        accel = (rest + damp + kr*rest_child[i]) # hack... needs mass=1
        n_angles[i] += velo[i] * dt + accel * dt**2 / 2
        n_velo[i] += accel * dt

        # Save our restoration force for parent
        assert parent[i] is None or parent[i] < i
        if parent[i] is not None:
            rest_child[parent[i]] += rest
    angles = n_angles
    velo = n_velo

    #rest_root = k * (preferred - angles[0])
    #velo[0] += (kr * rest_root + rest) * dt
    #angles[0] += velo[0] * dt

In [154]:
f = plt.figure()
lim = 3
plt.xlim(-lim, lim)
plt.ylim(0, lim)

angles = history
'''
angles = [
    (np.pi/2, np.pi/2),
    (1.2*np.pi/2, np.pi/2),
    (np.pi/2, np.pi/2),
    (0.7*np.pi/2, np.pi/2),
    (np.pi/2, np.pi/2),
]
'''
lines = [plt.plot([], [], 'r-')[0] for _ in preferred]
plt.xticks([])
plt.yticks([])

def update_line(t):
    points = [
        np.array([np.cos(a), np.sin(a)])
        for a in angles[t]
    ]
    for b in range(len(preferred)):
        par = np.array([0, 0])
        if parent[b] is not None:
            par = points[parent[b]]
            points[b] += par
        lines[b].set_data(*zip(par, points[b]))
    return []

line_ani = animation.FuncAnimation(f, update_line, len(angles), fargs=(),
                                   interval=100*dt, blit=True)
plt.close()
HTML(line_ani.to_html5_video())

# some sample dynamics code

In [30]:
#https://www.toptal.com/game/video-game-physics-part-i-an-introduction-to-rigid-body-dynamics
class Particle(object):
    def __init__(self):
        self.position = np.array([random.randint(0, 10), random.randint(0, 10)], dtype=np.float)
        self.velocity = np.array([0., 0])
        self.mass = 1

    def step(self, force, dt):
        acceleration = force / self.mass
        self.velocity += acceleration * dt
        self.position += self.velocity * dt

    def __repr__(self):
        return f'Particle(position={self.position}, velocity={self.velocity}, mass={self.mass})'


In [31]:
def sim(time=10, dt=1):
    curr = 0
    particles = [Particle()]
    while curr < time:
        for particle in particles:
            particle.step(np.array([0, -9.81 * particle.mass]), dt)
        curr += dt
        print(particles)
sim()

[Particle(position=[10.    0.19], velocity=[ 0.   -9.81], mass=1)]
[Particle(position=[ 10.   -19.43], velocity=[  0.   -19.62], mass=1)]
[Particle(position=[ 10.   -48.86], velocity=[  0.   -29.43], mass=1)]
[Particle(position=[ 10.  -88.1], velocity=[  0.   -39.24], mass=1)]
[Particle(position=[  10.   -137.15], velocity=[  0.   -49.05], mass=1)]
[Particle(position=[  10.   -196.01], velocity=[  0.   -58.86], mass=1)]
[Particle(position=[  10.   -264.68], velocity=[  0.   -68.67], mass=1)]
[Particle(position=[  10.   -343.16], velocity=[  0.   -78.48], mass=1)]
[Particle(position=[  10.   -431.45], velocity=[  0.   -88.29], mass=1)]
[Particle(position=[  10.   -529.55], velocity=[  0.  -98.1], mass=1)]


In [19]:
f = plt.figure()
data = np.random.rand(2, 25)
l, = plt.plot([], [], 'r-')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.xlabel('x')
plt.title('test')
def update_line(num, data, line):
    line.set_data(data[..., :num])
    return []

line_ani = animation.FuncAnimation(f, update_line, 25, fargs=(data, l),
                                   interval=100, blit=True)
plt.close()
HTML(line_ani.to_html5_video())